In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType
#from rateFunction import *
spark = SparkSession.builder.appName("Ngram_buzzwords").master("spark://192.168.1.76:7077").getOrCreate()
sc = spark.sparkContext

In [2]:
def yearRate(word):
    if len(word) == 0:
        return [('0',0,0.0)]
    
    if len(word) == 1:
        return [(word[0][0], 0, 0.0)]
    
    wordArray=[]
    for i in range(len(word)-1):
        #try:
        yearPair = str(word[i][0])+"-"+str(word[i+1][0])
        diff = int(word[i+1][1])-int(word[i][1])
        diffProcent = (float(word[i+1][1])/float(word[i][1]))*100    
        wordArray.append((yearPair,diff,diffProcent))
        #except:
            #return [('0',0,0.0)]
    return wordArray

def sortBuzzWords(buzzwords, index_value):
    return sorted(buzzwords, key=lambda x: x[index_value], reverse=True)

In [3]:
df = spark.read.csv('../../data/eng-gb-all-only-letters.csv', sep="\t", header=False)

In [4]:
df = df.withColumnRenamed('_c0', 'ngram')
df = df.withColumnRenamed('_c1', 'year')
df = df.withColumnRenamed('_c2', 'match_count')
df = df.withColumnRenamed('_c3', 'page_count')
df = df.withColumnRenamed('_c4', 'volume_count')

In [5]:
#df.count()

In [6]:
rdd = df.rdd.map(list)

In [7]:
def splitAndMakeNgramKey(row):
    elements = row[0].split(",")
    return(elements[0], elements[1:len(elements)])

rdd = rdd.map(splitAndMakeNgramKey)

In [8]:
#rdd.take(10)

In [9]:
#rdd2 = sc.parallelize(rdd.take(1000000))

In [10]:
def groupData(a,b):
    if isinstance(a[0], (list,)):
        return a+[b]
    else:
        return [a]+[b]

rdd2 = rdd.reduceByKey(groupData)

In [ ]:
rdd2.take(100)

In [11]:
rdd3 = rdd2.map(lambda row: (row[0], yearRate(row[1])))

In [12]:
rdd_count = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 1)))

In [13]:
rdd_percent = rdd3.map(lambda row: (row[0], sortBuzzWords(row[1], 2)))

In [ ]:
rdd_count.take(10)

In [ ]:
rdd_percent.take(10)

In [ ]:
#thresholdProcent = 1000
#rdd_filter = rdd3.filter(lambda x:x[] )
def threshHold:
    

In [ ]:
rdd_count2 = rdd_count.sortBy(lambda row: row[1][0][1], ascending=False)

In [ ]:
rdd_percent2 = rdd_percent.sortBy(lambda row: row[1][0][2], ascending=False)

In [ ]:
rdd_count2.take(10)

In [ ]:
rdd_percent2.take(10)

In [ ]:
#rdd3.coalesce(1).saveAsTextFile("Words-with-year-list.txt")